# 2019年度　パターン認識最終課題

パターン認識最終課題では、銀行の顧客に関するデータセットを使って、顧客の婚姻状況を予測するモデルを構築してもらいます。

学籍番号と氏名を記入してください。
記入例：1811000 先端　太郎

1911328 TAO XIANGKUN

## 課題１: 色々なモデルを試してみよう！
まずは、講義で扱った様々な識別器を試してみましょう。
以下に示すロジスティック回帰の例を参考に、指示に従いSVM、K-近傍法、決定木、ランダムフォレストの実装を行ってもらいます。

In [1]:
from IPython.display import display
import pandas as pd
from pandas import DataFrame, Series
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline  
import matplotlib.pyplot as plt
import numpy as np
RAND_SEED = 42

### Step1 データの読み込み
まずは、データを読み込みます。

In [3]:
df_train_data = pd.read_csv('train.csv')

今回は、この中で、marital（婚姻状況）を予測するモデルの構築を行ってもらいます。

### Step 2　使用する列の抽出
次に、使用する列だけを取り出します。　今回はmaritalと、予測に使えそうな age, housing を取り出します。

In [4]:
y_ = df_train_data.marital# ラベルデータ
y_.value_counts()

married     12141
single       5605
divorced     2254
Name: marital, dtype: int64

In [5]:
DataFrame(y_).head(10)

,marital
0,married
1,married
2,divorced
3,married
4,married
5,single
6,married
7,single
8,divorced
9,married


ラベルデータの上位10件を表示しました。

In [6]:
X_ = df_train_data[['age', 'housing']]

In [7]:
X_.head(10)

,age,housing
0,58,yes
1,38,yes
2,81,yes
3,49,yes
4,38,no
5,47,yes
6,33,no
7,31,yes
8,31,no
9,51,yes


特徴量データの上位10件を表示しました。

### Step3　特徴量データの前処理
識別器での学習を行う前に、特徴量を適切な形へ変換する必要があります。
その過程を見ていきましょう。

まず、欠損値を含むデータの扱いですが、本課題では、簡単のため単純に削除する手法を取ります。

In [8]:
df_temp = pd.concat([y_, X_], axis=1).dropna(how='any')# 欠損値を含むデータの削除
df_temp.index = range(len(df_temp)) 
y = df_temp.marital
X = df_temp[['age', 'housing']]

次に、housing 列は文字列であるため、数値に変換してあげる必要があります。ここでは、yes = 2, unknown = 1, no = 0 として変換します。

In [9]:
X.tail(10)

,age,housing
19990,33,yes
19991,41,yes
19992,54,no
19993,44,yes
19994,59,no
19995,24,no
19996,42,yes
19997,31,no
19998,25,yes
19999,33,no


In [10]:
housing_le = LabelEncoder()
X.housing = housing_le.fit_transform(X.housing)

In [11]:
X.tail(10)

,age,housing
19990,33,2
19991,41,2
19992,54,0
19993,44,2
19994,59,0
19995,24,0
19996,42,2
19997,31,0
19998,25,2
19999,33,0


これで、数値データに置き換えられました。


最後に、それぞれの特徴量の数値の変動量に差による影響力の差を排除するため、正規化を行います。

In [12]:
mm_scaler = MinMaxScaler()

In [13]:
X.age = DataFrame(mm_scaler.fit_transform(DataFrame(X.age.copy())))
X.housing = DataFrame(mm_scaler.fit_transform(DataFrame(X.housing.copy())))

In [14]:
X.tail(10)

,age,housing
19990,0.197531,1.0
19991,0.296296,1.0
19992,0.456790,0.0
19993,0.333333,1.0
19994,0.518519,0.0
19995,0.086420,0.0
19996,0.308642,1.0
19997,0.172840,0.0
19998,0.098765,1.0
19999,0.197531,0.0


これで、正規化が出来ました。

### Step 4　識別器の実装
ロジスティック回帰の例を参考に、SVM、K-近傍法、決定木、ランダムフォレストを実装しましょう。

#### ロジスティック回帰
ロジスティック回帰の例を示します。実装にはscikit-learn(http://scikit-learn.org/stable/) の関数を使用します。

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 訓練データとテストデータに分割
default_acc = {}

In [16]:
logistic_regression = LogisticRegression() # ロジスティック回帰モデルの作成
logistic_regression.fit(X_train, y_train) # 訓練データを使って学習
y_pred = logistic_regression.predict(X_test) # 予測値の取得

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['logistic_regression'] = acc

accuracy:0.662


上記の LogisticRegression( ) の部分を書き換えて、SVM、K-近傍法、決定木、ランダムフォレストを実装しましょう。

#### SVM
ここで、SVM (SVC) の実装を行ってください。特徴量も自由に選択して構いません。

In [17]:
svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['SVC'] = acc

accuracy:0.6625


#### K-近傍法

ここで、K-近傍法 (KNeighborsClassifier) の実装を行ってください。特徴量も自由に選択して構いません。

In [18]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['KNN'] = acc

accuracy:0.577


#### 決定木

ここで、決定木 (DecisionTreeClassifier) の実装を行ってください。特徴量も自由に選択して構いません。

In [19]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['decision_tree'] = acc

accuracy:0.658


#### ランダムフォレスト

ここで、ランダムフォレスト (RandomForestClassifier) の実装を行ってください。特徴量も自由に選択して構いません。

In [20]:
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['random_forest'] = acc

accuracy:0.66


#### Additional module: Bagging KNN classifier

In [21]:
from sklearn.ensemble import BaggingClassifier
bagging_knn = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
bagging_knn.fit(X_train, y_train)
y_pred = bagging_knn.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['Bagging KNN'] = acc

accuracy:0.652


#### Additional module: Naive Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['naive_bayes'] = acc

accuracy:0.6605


#### Additional module: XGBoost classifier

In [23]:
import xgboost
xgb = xgboost.XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['xgboost'] = acc

accuracy:0.6625


#### Additional module: GBDT

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier()
gbdt.fit(X_train, y_train)
y_pred = gbdt.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['GBDT'] = acc

accuracy:0.661


#### Additional module: Perceptron

In [25]:
from sklearn.linear_model import Perceptron
perceptron = Perceptron(tol=1e-3, random_state=RAND_SEED)
perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['Perceptron'] = acc

accuracy:0.289


#### Additional module: SGD

In [26]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(max_iter=1000, tol=1e-3)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['SGD'] = acc

accuracy:0.6605


#### Additional module: Voting classifier 

In [27]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier(estimators={
    ('gbdt', gbdt),
    ('random_forest', random_forest),
    ('logistic_regression', logistic_regression),
    #('nb', nb),
    ('bagging_knn', bagging_knn),
    ('decision_tree', decision_tree),
    ('knn', knn),
    ('xgb', xgb),
    #('svm', svm),
    #('perceptron', perceptron),
    #('sgd', sgd),
}, voting='soft')
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)

acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f'accuracy:{acc}') # 精度の計算
default_acc['Voting'] = acc
print("Accurate for default classifiers:")
default_acc

accuracy:0.663
Accurate for default classifiers:


{'logistic_regression': 0.662,
 'SVC': 0.6625,
 'KNN': 0.577,
 'decision_tree': 0.658,
 'random_forest': 0.66,
 'Bagging KNN': 0.652,
 'naive_bayes': 0.6605,
 'xgboost': 0.6625,
 'GBDT': 0.661,
 'Perceptron': 0.289,
 'SGD': 0.6605,
 'Voting': 0.663}

# 課題２: モデルの改善を行おう！
課題1で作成した各モデルについて、どうすればさらに良い精度を出すことができるのか、パラメータの調整や特徴量の選定などを行いましょう。
最後に、この課題において最善と考える特徴量とモデルを述べ、その理由も示してください。

まず、選定した特徴量を用いて、課題１で行ったStep1〜Step3 を行いましょう。ただし、Step 2〜Step 3の処理は、preprocessingという関数で定義してください。ここで定義した関数を用いて評価を行います。
引数には、読み込んだデータを指定してください。返り値には、前処理を行った後の選定した特徴量 ( X )とmarital ( y )を指定してください。

In [28]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer

In [43]:
### Step 1　　データの読み込み
#raw_data = pd.read_csv('train.csv')
#raw_data.dropna(how='any')
#train_data, test_data = train_test_split(raw_data, test_size=0.1, random_state=RAND_SEED)
#raw_data.hist(bins=50, figsize=(20,15))
raw_data = pd.read_csv('train.csv')
train_data, test_data = train_test_split(raw_data, test_size=0.1, random_state=RAND_SEED)
num_attrs = ["age"]
cat_attrs = ["job", "education", "default", "housing", "loan", "y"]
feature_attrs = num_attrs + cat_attrs
target = ["marital"]

num_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    #StandardScaler(),
    MinMaxScaler(),
)

preproc_onehot = make_column_transformer(
    (num_pipeline, num_attrs),
    (OneHotEncoder(), cat_attrs),
)

preproc_ordinal = make_column_transformer(
    (num_pipeline, num_attrs),
    (OrdinalEncoder(), cat_attrs),
)

full_pipeline = make_pipeline(
    preproc_onehot,
    sgd,
    #PolynomialFeatures(degree=2),
    #GaussianNB(),
    #xgboost.XGBClassifier(random_state=RAND_SEED),
    #LogisticRegression(random_state=RAND_SEED),
    #SVC(),
    #GradientBoostingClassifier()
    #voting
    #DecisionTreeClassifier(),
    #KNeighborsClassifier(),
)

full_pipeline.fit(train_data[feature_attrs], train_data[target])
print(f"acc: {accuracy_score(full_pipeline.predict(test_data[feature_attrs]), test_data[target]):.4f}")

acc: 0.6635


In [30]:
### 課題１で行ったStep 2〜Step 3を関数として定義してください。
def preprocessing(df_train_data):
    ### Step 2　　使用する列の抽出
    y_ = df_train_data.marital# ラベルデータ
    X_ = df_train_data[['age', 'housing']]
    
    ### Step 3　　特徴量データの前処理
    # 欠損値を含むデータの削除
    df_temp = pd.concat([y_, X_], axis=1).dropna(how='any')
    df_temp.index = range(len(df_temp)) 
    y = df_temp.marital
    X = df_temp[['age', 'housing']]
    
    # 文字列を数値データへ変換
    housing_le = LabelEncoder()
    X.housing = housing_le.fit_transform(X.housing)
    
    # 数値データの正規化
    mm_scaler = MinMaxScaler()
    X.age = DataFrame(mm_scaler.fit_transform(DataFrame(X.age.copy())))
    X.housing = DataFrame(mm_scaler.fit_transform(DataFrame(X.housing.copy())))
    
    return X, y

上記で定義した関数を実行します。

In [31]:
X, y = preprocessing(df_train_data)

次に、この課題における最善なモデルを構築しましょう。課題１で使用したモデル以外を使用しても構いません。
下記は、ロジスティック回帰の場合の例です。

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 訓練データとテストデータに分割

In [33]:
logistic_regression = LogisticRegression() # ロジスティック回帰モデルの作成
logistic_regression.fit(X_train, y_train) # 訓練データを使って学習
y_pred = logistic_regression.predict(X_test) # 予測値の取得
print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test))) # 精度の計算

accuracy:0.65


### 考察１
この課題において最善と考える特徴量を述べ、その理由も示してください。

（ここに記入してください）

### 考察２
この課題において最善と考えるモデルを述べ、その理由も示してください。

（ここに記入してください）

最後に、以下の best_model に構築したモデルを記入してください。この best_model を使用して精度評価を行います。

In [34]:
best_model = logistic_regression     # この例では、ロジスティック回帰を選択したので、logistic_regression を記入しています。
pickle.dump(best_model, open('best_model.pkl','wb'))

### --------評価用--------
ここから下は評価用となります。上の欄にモデルを記入した後は、配布資料に書いてある手順に沿ってファイルを提出してください。

In [35]:
# df_test_data = pd.read_csv('bank_test.csv')
# preprocessing()
# X_test = X
# y_test = y

In [36]:
# best2_model = pickle.load(open('best_model.pkl', 'rb'))
# y_pred =best2_model.predict(X_test)
# print('accuracy:{}'.format(accuracy_score(y_pred=y_pred, y_true=y_test)))